## Importamos la clase Vec

In [1]:
from vec import Vec

In [2]:
#crear un diccionario para cada archivo csv
id_movies = {(i.strip().split(','))[0] for i in list(open('movies.csv'))}
id_user = {(i.strip().split(','))[0] for i in list(open('ratings.csv'))}
id_user.remove('user_id')
users = {}

#ciclo que da vueltas para ir guardando el id del usuario y los rating de las peliculas 
#en la clase vector
for i in id_user:
    dic = {}
    archivo = open('ratings.csv', 'r')
    archivo = archivo.readlines()
    
    for x in archivo:
        x = x.strip('\n')
        x = x.split(',')
        if x[0] == i:
            dic[x[1]] = int(x[2]) 
    users[i] = Vec(id_movies,dic)

In [3]:
#probamos que el diccionario funcione buscando el rating de la pelicula de id 2 del usuario de id 943
#este resultando ser 5
users['943']['2']

5

In [4]:
id_movies.remove('movie_id')
movies = {}

#ciclo que recorre todas las peliculas y las va guardando en la clase vector
#con sus respectivos rating de cada usuario
for i in id_movies:
    dic = {}
    archivo = open('ratings.csv', 'r')
    archivo = archivo.readlines()
    
    for x in archivo:
        x = x.strip('\n')
        x = x.split(',')
        if x[1] == i:
            dic[x[0]] = int(x[2]) 
    movies[i] = Vec(id_user,dic)

In [36]:
#probamos que funcione el diccionario de las peliculas buscando la pelicula de id 922 y el rating 
#que le dio el usuario de id 883 que en este caso es 5
movies['922']['883']


5

## Código pregunta c)

In [37]:
# Creamos una función para encontrar la similaridad entre los usuarios

def vecinos(users, user_id, k):
    
    user_principal = users[str(user_id)]
    
    #Creamos dos listas, una para los usuarios que queremos comparar con el user_principal y otra con la similitud. 
    #Para las dos listas cada índice corresponde al mismo usuario, por ejemplo para un número x lista_usuarios[x] 
    #es el usuario y lista_similitudes[x] corresponde a su similitud con el user_principal.
    
    lista_usuarios = []
    lista_similitudes = []
    
    for x in users:
        
        #Este if es para evitar calcular la similitud del user_principal consigo mismo
        
        if users[str(x)] != user_principal:
            
            #Estas son las operaciones para calcular la similitud
            
            raiz_1 = (users[str(x)] * users[str(x)])**(1/2)
            raiz_2 = (user_principal * user_principal)**(1/2)
                        
            # Hacemos append a ambas listas con sus respectivos contenidos
            
            lista_similitudes.append((user_principal * users[str(x)])/(raiz_1*raiz_2))
            lista_usuarios.append(str(x))
            
    # Esta es la lista que contendrá las tuplas 
    lista_final = []
    
    
    for x in range(len(lista_usuarios)):
        
        #Creamos las tuplas con su respectivo usuario y similitud
        
        lista = []
        lista.append(lista_usuarios[x])
        lista.append(lista_similitudes[x])
        lista = tuple(lista)
        lista_final.append(lista)
    
    #Ordenamos la lista final para obtener los usuarios con mayor similitud al user_principal
    lista_final = sorted(lista_final, key = lambda x: x[1], reverse=True)
    
    lista_final_final = []
    
    #Tomamos los k usuarios de la lista_final
    
    for x in range(k):
        
        
        lista_final_final.append(lista_final[x])
        
    return lista_final_final
    

### Este es un ejemplo para probar la función

In [38]:
vecinos(users, 343, 5)

[('823', 0.48649820384918496),
 ('864', 0.4834252361218437),
 ('916', 0.45534171088502),
 ('664', 0.44806881497240225),
 ('933', 0.44053112736963473)]

## Pregunta d)

In [39]:
def rating_user_100(users, user_id, k):
    
    #Usamos la función vecinos para encontrar los k usuarios con mayor similitud al usuario que nos piden(que en este caso va a ser el usuario 100).
    
    lista_usuarios_cercanos = vecinos(users, user_id, k)
    
    #Luego verificamos los usuarios que hayan visto la película "286".
    #Para ello creamos una lista en la que los agregaremos.
    
    usuarios_con_rating = []
    
    for x in lista_usuarios_cercanos:
        
        #Aquí verificamos que la hayan visto
        
        if users[x[0]]['286'] != 0:
            
            usuarios_con_rating.append(x[0])
    
    # Retornamos la lista
    return usuarios_con_rating
    

In [40]:
print(len(rating_user_100(users, 100, 15)))

#Para sacar el promedio creamos una variable a la cual le vamos sumando todos
#los puntajes que le dieron a la película los usuarios de la lista.

promedio = 0

for x in rating_user_100(users, 100, 15):
    
    promedio += int(users[x]['286'])

#Calculamos el promedio

promedio = (promedio/len(rating_user_100(users, 100, 15)))
promedio

12


3.8333333333333335

## Código pregunta e)

In [41]:
#Creamos una variable e, que contenga una lista con los 30 usuarios con mayor similaridad al usuario "400".

e = vecinos(users, 400, 30)

#Además creamos una variable combinacion_convexa que contenga el vector promedio resultante de la combinación convexa 
#entre los vectores de cada usuario de la lista "e".

combinacion_convexa = 0

for x in e:
    
    combinacion_convexa += users[x[0]]
    
combinacion_convexa = combinacion_convexa/30

#Creamos una lista que contenga los ratings promedios de cada película en orden decreciente.

lista_ratings = sorted(combinacion_convexa.f, key = lambda x:combinacion_convexa[x], reverse=True)

#Obtenemos los id de las 10 películas con mayor rating.

id_peliculas = []

for x in range(10):
    
    id_peliculas.append(lista_ratings[x])
    
id_peliculas

#Abrimos el archivo 'movies.csv' para obtener los nombres de las películas.

archivo = open('movies.csv', 'r')

nombres_peliculas = []

#Nos dimos cuenta que había algunas películas que tenían una coma en su nombre, 
#por lo que dependiendo de si la tenían o no, le haciamos un split() diferente.


for x in archivo:
    
    z = x.split(',')
    
    if z[0] in id_peliculas:
        
        if '"' in x:
            
            z = x.split('"')
            
            nombres_peliculas.append(z[1])
            
        else: 
            
            nombres_peliculas.append(z[1].strip('\n'))

#Cerramos el archivo y hacemos print a las 10 películas.
            
#archivo.close()

nombres_peliculas

['Contact (1997)',
 'Full Monty, The (1997)',
 'English Patient, The (1996)',
 'Scream (1996)',
 'Liar Liar (1997)',
 'Air Force One (1997)',
 'Titanic (1997)',
 'Conspiracy Theory (1997)',
 'Saint, The (1997)',
 'Scream 2 (1997)']